In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
import pandas as pd

In [3]:
import sys
sys.path.insert(0, '../../rna_ss/')
from utils import db2pairs
from local_struct_utils import one_idx2arr, sort_pairs, LocalStructureParser, make_target

/Users/alicegao/anaconda2/envs/plot_py3/lib/python3.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/Users/alicegao/anaconda2/envs/plot_py3/lib/python3.7/site-packages/Bio/Alphabet/__init__.py:27: PendingDeprecationWarning: We intend to remove or replace Bio.Alphabet in 2020, ideally avoid using it explicitly in your code. Please get in touch if you will be adversely affected by this. https://github.com/biopython/biopython/issues/2046
  PendingDeprecationWarning,


In [4]:
import plotly.graph_objects as go

In [5]:
# df = pd.read_pickle('../../rna_ss/data_processing/rna_cg/data/rfam.pkl')  # verified
# df = pd.read_pickle('../../rna_ss/data_processing/e2efold/data/rnastralign.pkl.gz')  # verified
# df = pd.read_pickle('../../rna_ss/data_processing/e2efold/data/archiveII.pkl.gz')  # verified
df = pd.read_pickle('../../rna_ss/data_processing/spot_rna/bp_rna/data/bp_rna.pkl.gz')

In [6]:
df.head()

,seq_id,data_partition,seq,len,one_idx,verified
0,RFAM_22522,VL0,GGAGCGAUUGGACCACCUUGCAGUUUCAUGGCACACUCUCUCCAUU...,169,"[[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, ...",True
1,CRW_34381,VL0,GCGGGUAUAGCUCAGUUGGUAGAGCGUGGUCCUUCCAAGUCCAAUG...,73,"[[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ...",True
2,CRW_44200,VL0,GGCGGGAUAGCUCAGCUGGUUAGAGCGCAUGAUUCAUAAUCAUGAG...,77,"[[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 26, 27, ...",True
3,RFAM_6181,VL0,GAAUACGCGCAAAAUGUUAUCAUCCCUAUUUCUCUAUAGAAUUUAU...,76,"[[20, 23, 25, 27, 48, 49, 50, 51, 52, 53, 54],...",True
4,RFAM_22751,VL0,UCACGAACAUGGCAAACUACAUGAAAAUGUAGGACGCAAAGCUUCC...,83,"[[11, 12, 16, 17, 18, 19, 20, 21, 39, 40, 41, ...",True


In [7]:
df.tail()

,seq_id,data_partition,seq,len,one_idx,verified
13414,tmRNA_686,TS0,GGGGCCGAUCCGGAUUCGACGUGGGUCGCGAAACAGUGCAGGGCAU...,366,"[[48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 112,...",True
13415,RFAM_1793,TS0,GAGAAUAUUGUUUAAUGGUAAAACAGUUGCCUUUUAAGCAUCCCAU...,72,"[[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, 27,...",True
13416,RFAM_38360,TS0,AAAGUCCCUGACGACCAAGACCUAGCGAGUCAUGUUGUCCCCGAGU...,224,"[[11, 12, 13, 14, 15, 18, 19, 20, 38, 39, 40, ...",True
13417,RFAM_28378,TS0,UGUUAUGGCAGUGUGCGUGGGAGAACCGCAAGGUUCGCCGUUUCUU...,86,"[[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1...",True
13418,RFAM_2828,TS0,CAUUGCAGGCAACCGAGCCAGCGAUGGUUAGGAAAGUCCAUGCUAG...,337,"[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17,...",True


In [8]:
# idx = 6
# idx = 286
# idx = 18
idx = 500
pairs, structure_arr = one_idx2arr(df.iloc[idx].one_idx, len(df.iloc[idx].seq), remove_lower_triangular=True)
pairs = sort_pairs(pairs)
parser = LocalStructureParser(pairs)

In [9]:
df.iloc[idx].seq_id

'RFAM_22978'

In [10]:
len(df)

13419

In [11]:
# # debug - sanity check
# for row_idx, row in df.iterrows():
#     pairs, structure_arr = one_idx2arr(row.one_idx, len(row.seq), remove_lower_triangular=True)
#     pairs = sort_pairs(pairs)
#     parser = LocalStructureParser(pairs)
#     print(row_idx, row.seq_id, len(row.seq), len(parser.local_structure_bounding_boxes))

In [12]:
target_vals, target_mask = make_target(structure_arr, parser.local_structure_bounding_boxes)

In [18]:
print(target_vals.shape)
print(target_mask.shape)

(130, 130, 5)
(130, 130)


In [24]:
# px.imshow(target_vals[:, :, 4])
px.imshow(target_mask)

In [14]:
# bounding boxes
# note that the value used for visualization is 0.5 pixel shifted (and expanded) so that it surrounds the pixels


fig = px.imshow(structure_arr)

name2color = {
    'stem': 'red',
    'bulge': 'cyan',
    'internal_loop': 'purple',
    'pesudo_knot': 'magenta',
    'hairpin_loop': 'white',
    
}

for ls in parser.local_structure_bounding_boxes:
    (x0, y0), (wx, wy), name = ls
    fig.add_shape(
        type='rect',
        y0=x0-0.5, y1=x0+wx-0.5, x0=y0-0.5, x1=y0+wy-0.5,   # image plot axis is swaped
        xref='x', yref='y',
        line_color=name2color[name]
    )


print(df.iloc[idx].seq)

fig.update_layout(
    title=df.iloc[idx].seq_id,
    font=dict(
        size=12,
    )
)

fig.show()


CCCGCCGAGGGGCGCUGCGACCAUGGACCUGCGGCCCGAGCCGCGAAGCCGCGCACACGUCCGUGCGCGCCUGCGAUCCCUGGCCAGGCUCGGCAGGGCGUUCCUUCAGCGACAACGGCGCCCGGACCAA


In [15]:
# todo debug - validate local structure?
for ls in parser.local_structure_bounding_boxes:
    (x0, y0), (wx, wy), name = ls
    print(*ls)
    tmp_data = structure_arr[x0:x0+wx, y0:y0+wy]
    print(tmp_data)
    print('')

(1, 89) (7, 7) stem
[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]

(20, 77) (6, 6) stem
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]

(7, 82) (14, 8) internal_loop
[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]

(25, 25) (53, 53) hairpin_loop
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

